In [1]:
import mysql.connector
import random 

mydb=mysql.connector.connect(
host="localhost",
user="root",
password="system",
database="hotel")
mydb

In [3]:
class Hhotel:
    def __init__(self):
        print("\n\n***** WELCOME TO STAR HOTEL *****\n")
        self.roomno = list(range(101, 500))  # Room numbers from 101 to 499
        self.generated_urns = set()

    def is_room_occupied(self, room_no):
        """
        Check if a room is already assigned to any customer in the Hotel.
        """
        sql = "SELECT * FROM customerdata WHERE roomno = %s"
        mycursor = mydb.cursor(dictionary=True)
        mycursor.execute(sql, (room_no,))
        result = mycursor.fetchone()
        return result is not None

    def get_input(self, prompt):
        """
        Get user input with an option to cancel.
        """
        while True:
            user_input = input(prompt).strip()
            if user_input.lower() == "cancel":
                return "cancel"
            elif user_input:
                return user_input
            else:
                print("Input cannot be empty. Please try again or type 'cancel' to go back.")

    def custdetails(self):
        # Ensure that the required fields are not null
        name = self.get_input("\nEnter your Name (or type 'cancel' to go back): ")
        if name == "cancel": return

        mobileno = self.get_input("\nEnter your Mobile No. (or type 'cancel' to go back): ")
        if mobileno == "cancel": return
        while not mobileno.isdigit() or len(mobileno) != 10:
            print("Please enter a valid 10-digit mobile number.")
            mobileno = self.get_input("\nEnter your Mobile No. (or type 'cancel' to go back): ")
            if mobileno == "cancel": return

        checkin = self.get_input("\nEnter your Check-in Date (DD/MM/YYYY, or type 'cancel' to go back): ")
        if checkin == "cancel": return
        while True:
            try:
                checkin_date = datetime.datetime.strptime(checkin, "%d/%m/%Y").date()
                break
            except ValueError:
                print("Invalid date format. Please use DD/MM/YYYY.")
                checkin = self.get_input("\nEnter your Check-in Date (DD/MM/YYYY, or type 'cancel' to go back): ")
                if checkin == "cancel": return

        checkout = self.get_input("\nEnter your Check-out Date (DD/MM/YYYY, or type 'cancel' to go back): ")
        if checkout == "cancel": return
        while True:
            try:
                checkout_date = datetime.datetime.strptime(checkout, "%d/%m/%Y").date()
                break
            except ValueError:
                print("Invalid date format. Please use DD/MM/YYYY.")
                checkout = self.get_input("\nEnter your Check-out Date (DD/MM/YYYY, or type 'cancel' to go back): ")
                if checkout == "cancel": return

        address = self.get_input("\nEnter your Address (or type 'cancel' to go back): ")
        if address == "cancel": return

        print("Select room type:")
        print("1. Press S for Single Bed room. (1000 per night)")
        print("2. Press D for Double Bed room. (1500 per night)")
        print("3. Press DX for Deluxe room. (3000 per night)")
        roomtype = input("Room Type: ").lower()

        room_no = None
        urn = None

        if roomtype == "s":
            while self.roomno:
                candidate_room = self.roomno.pop(0)
                if not self.is_room_occupied(candidate_room):
                    room_no = candidate_room
                    break
            if room_no:
                print(f"Single Bed Room {room_no} allocated.")
                urn = random.randint(1000, 9999)
                while urn in self.generated_urns:
                    urn = random.randint(1000, 9999)
                self.generated_urns.add(urn)
                print(f"URN: {urn}")
            else:
                print("No Single Bed room Available")
                return

        elif roomtype == "d":
            while self.roomno:
                candidate_room = self.roomno.pop(0)
                if not self.is_room_occupied(candidate_room):
                    room_no = candidate_room
                    break
            if room_no:
                print(f"Double Bed Room {room_no} allocated.")
                urn = random.randint(1000, 9999)
                while urn in self.generated_urns:
                    urn = random.randint(1000, 9999)
                self.generated_urns.add(urn)
                print(f"URN: {urn}")
            else:
                print("No Double Bed rooms available.")
                return
        elif roomtype == "dx":
            while self.roomno:
                candidate_room = self.roomno.pop(0)
                if not self.is_room_occupied(candidate_room):
                    room_no = candidate_room
                    break
            if room_no:
                print(f"Deluxe Room {room_no} allocated.")
                urn = random.randint(1000, 9999)
                while urn in self.generated_urns:
                    urn = random.randint(1000, 9999)
                self.generated_urns.add(urn)
                print(f"URN: {urn}")
            else:
                print("No Deluxe rooms available.")
                return
        else:
            print("Invalid room type selected.")
            return

        # Insert room booking into database
        sql = "INSERT INTO customerdata(roomno, name, mobileno, urn, checkin, checkout, address) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        val = (room_no, name, mobileno, urn, checkin, checkout, address)
        mycursor = mydb.cursor()
        mycursor.execute(sql, val)
        mydb.commit()
        print("Room booked successfully.")

    def display_customers(self):
        # Display all customers
        print("\n***** Customer Details *****")
        sql = "SELECT * FROM customerdata"
        mycursor = mydb.cursor(dictionary=True)
        mycursor.execute(sql)
        customers = mycursor.fetchall()

        for customer in customers:
            print(f"Room No: {customer['roomno']}")
            print(f"Name: {customer['name']}")
            print(f"Address: {customer['address']}")
            print(f"Mobile No.: {customer['mobileno']}")
            print(f"checkin: {customer['checkin']}")
            print(f"Check-out: {customer['checkout']}")
            print("-" * 30)

    def show_data(self):
        urn_no = input("\nEnter the URN NO. to search (or type 'cancel' to go back): ").strip()
        if urn_no.lower() == "cancel":
            return

        sql = "SELECT * FROM customerdata WHERE urn = %s"
        mycursor = mydb.cursor(dictionary=True)
        mycursor.execute(sql, (urn_no,))
        customer = mycursor.fetchone()

        if customer:
            print("\n***** Customer Details *****")
            print(f"Room No: {customer['roomno']}")
            print(f"Name: {customer['name']}")
            print(f"Address: {customer['address']}")
            print(f"Mobile No.: {customer['mobileno']}")
            print(f"checkin: {customer['checkin']}")
            print(f"Check-out: {customer['checkout']}")
            print("-" * 30)
        else:
            print("\nCustomer with this URN not found.")

    def calculate_bill(self):
        # Asking for mobile number to get the bill
        phone_no = input("\nEnter the Mobile No. to calculate the bill (or type 'cancel' to go back): ").strip()
        if phone_no.lower() == "cancel":
            return

        # Searching for the customer by mobile number
        sql = "SELECT * FROM customerdata WHERE mobileno = %s"
        mycursor = mydb.cursor(dictionary=True)
        mycursor.execute(sql, (phone_no,))
        customer = mycursor.fetchone()

        if customer:
            # Parsing dates
            checkin_date = datetime.datetime.strptime(customer["checkin"], "%d/%m/%Y")
            checkout_date = datetime.datetime.strptime(customer["checkout"], "%d/%m/%Y")
            
            # Calculating number of nights
            num_nights = (checkout_date - checkin_date).days

            # Room type and rate
            room_no = customer["roomno"]
            if room_no < 110:
                room_rate = 1000  # Single Bed
            elif room_no < 210:
                room_rate = 1500  # Double Bed
            else:
                room_rate = 3000  # Deluxe Room

            # Calculating the total room charge
            room_charge = room_rate * num_nights

            
            service_charge = room_charge * 0.10  # 10% of room charge
            
            tax = room_charge * 0.18  # 18% of room charge
            
            cleaning_fee = room_charge * 0.05  # 5% of room charge

            # Calculating the total bill
            total_bill = room_charge + service_charge + tax + cleaning_fee

            print("\n***** Bill Details *****")
            print(f"Room Type: {'Single Bed' if room_no < 110 else 'Double Bed' if room_no < 210 else 'Deluxe Room'}")
            print(f"Room No: {customer['roomno']}")
            print(f"Number of Nights: {num_nights}")
            print(f"Room Charge: {room_charge}")
            print(f"Service Charge (10%): {service_charge}")
            print(f"Tax (18%): {tax}")
            print(f"Cleaning Fee (5%): {cleaning_fee}")
            print(f"Total Bill: {total_bill}")
        else:
            print("\nCustomer with this mobile number not found.")

    def menu(self):
        while True:
            print("\n***** MENU *****")
            print("1. Enter Customer Data")
            print("2. Show Room's Bill")
            print("3. Show Customer's Details and Bill")
            print("4. Show All Customers' Details")
            print("5. Exit")
            choice = input("Enter your choice: ")

            if choice == "1":
                self.custdetails()
            elif choice == "2":
                self.calculate_bill()
            elif choice == "3":
                self.show_data()
            elif choice == "4":
                self.display_customers()
            elif choice == "5":
                print("Exiting the system.")
                break
            else:
                print("Invalid choice. Please try again.")


hotel = Hhotel()
hotel.menu()




***** WELCOME TO STAR HOTEL *****


***** MENU *****
1. Enter Customer Data
2. Show Room's Bill
3. Show Customer's Details and Bill
4. Show All Customers' Details
5. Exit


Enter your choice:  1

Enter your Name (or type 'cancel' to go back):  Kunal

Enter your Mobile No. (or type 'cancel' to go back):  9625356336

Enter your Check-in Date (DD/MM/YYYY, or type 'cancel' to go back):  06/02/2025


NameError: name 'datetime' is not defined